# This is my first attempt on the MNIST dataset

In [4]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [6]:
#dataset already exists in tensorflow library
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

### Working with the Data

In [98]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

#creating validation data
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

#creating feature scaling fucntion
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

#scaling train, validation, and test data
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

#shuffling the data
buffer_size = 10000 #used only for big datasets
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

#setting validation and train data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

#setting batch size for gradient descent
batch_size = 100
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples) #we don't actually need to batch it, as we are only forward propagating on validation data. However, we use the batch method as the built in methods expect the data in batch form
test_data = scaled_test_data.batch(num_test_samples) #same logic as validation data

validation_inputs, validation_targets = next(iter(validation_data))

### Creating the Model

In [99]:
# 784 input nodes (28x28 pixels), 150 hidden nodes, 3 hidden layers, 10 output nodes (digits from 1 to 10)

In [107]:
input_size = 784
output_size = 10
hidden_layer_size = 150

#creating model layers
model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax') #softmax gives us output as probablity distribution
                            ]) 

### Optimizer and Loss Function

In [108]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']) #Note: strings are not case-sensitive. However, methods are!

### Model Training

In [110]:
num_epochs = 5

model.fit(train_data, epochs = num_epochs, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.0422 - accuracy: 0.9867 - val_loss: 0.0479 - val_accuracy: 0.9847
Epoch 2/5
540/540 - 1s - loss: 0.0359 - accuracy: 0.9882 - val_loss: 0.0475 - val_accuracy: 0.9848
Epoch 3/5
540/540 - 1s - loss: 0.0291 - accuracy: 0.9905 - val_loss: 0.0381 - val_accuracy: 0.9878
Epoch 4/5
540/540 - 1s - loss: 0.0250 - accuracy: 0.9916 - val_loss: 0.0254 - val_accuracy: 0.9922
Epoch 5/5
540/540 - 1s - loss: 0.0231 - accuracy: 0.9921 - val_loss: 0.0250 - val_accuracy: 0.9920


#### Highest validation accuraty 99.20%

### Testing the Model

In [111]:
test_loss, test_accuraty = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.0787 - accuracy: 0.9800


## Final model accuracy is 98.00%!